In [1]:
from plan_runner.environment import ManipStationEnvironment
import numpy as np
import torch
import time
import argparse
import os

import utils
import TD3

In [2]:
# Runs policy for X episodes and returns average reward
def evaluate_policy(policy, eval_episodes=10):
    avg_reward = 0.
    for i in xrange(eval_episodes):
        print("Test {}".format(i))
        obs = env.reset()
        done = False
        while not done:
            action = policy.select_action(np.array(obs))
            obs, reward, done, _ = env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print("Evaluation over %d episodes: %f" % (eval_episodes, avg_reward))
    print("---------------------------------------")
    return avg_reward

**Use this to visualize policies**

In [3]:
seed = 10
num_evals = 100
visualize = False

In [4]:
policy_name = "TD3"
env_name = "ManipStation"

file_name = "%s_%s_%s" % (policy_name, env_name, str(seed))
print("---------------------------------------")
print("Settings: %s" % (file_name))
print("---------------------------------------")

if not os.path.exists("./results"):
    exit()

env = ManipStationEnvironment(is_visualizing=visualize)
print("Starting in 1 second...")
time.sleep(1)
print("Starting!")

# Set seeds
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

state_dim = env.state_dim
action_dim = env.action_space.shape[0] 
max_action = float(env.action_space.high[0])

policy = TD3.TD3(state_dim, action_dim, max_action)
policy.load(file_name, directory="./pytorch_models")

evaluate_policy(policy, eval_episodes=num_evals)

---------------------------------------
Settings: TD3_ManipStation_10
---------------------------------------
Starting in 1 second...
Starting!
Test 0
Test 1
Test 2
Test 3
Test 4
Test 5
Test 6
Test 7
Test 8
Test 9
Test 10
Test 11
Test 12
Test 13
Test 14
Test 15
Test 16
Test 17
Test 18
Test 19
Test 20
Test 21
Test 22
Test 23
Test 24
Test 25
Test 26
Test 27
Test 28
Test 29
Test 30
Test 31
Test 32
Test 33
Test 34
Test 35
Test 36
Test 37
Test 38
Test 39
Test 40
Test 41
Test 42
Test 43
Test 44
Test 45
Test 46
Test 47
Test 48
Test 49
Test 50
Test 51
Test 52
Test 53
Test 54
Test 55
Test 56
Test 57
Test 58
Test 59
Test 60
Test 61
Test 62
Test 63
Test 64
Test 65
Test 66
Test 67
Test 68
Test 69
Test 70
Test 71
Test 72
Test 73
Test 74
Test 75
Test 76
Test 77
Test 78
Test 79
Test 80
Test 81
Test 82
Test 83
Test 84
Test 85
Test 86
Test 87
Test 88
Test 89
Test 90
Test 91
Test 92
Test 93
Test 94
Test 95
Test 96
Test 97
Test 98
Test 99
---------------------------------------
Evaluation over 100 episod

-32.26717648748514